In [ ]:
import os
import csv
import json
import pickle as pkl

wdir = '../results/hpsearch-2021-09-06/best/1_run_168/evalmockAllStates/step_0/'

with open(os.path.join(wdir, 'sim.pkl'), 'rb') as f:
    sim = pkl.load(f)

ts = []
with open(os.path.join(wdir, 'ActionsRewards.txt')) as f:
    for row in csv.reader(f, delimiter='\t'):
        ts.append(float(row[0]))
        
ts.append(ts[-1] + 50)

spkt = sim['simData']['spkt']
spkid = sim['simData']['spkid']

In [ ]:
input_pop = list(range(0, 80))
space_between = 50.0 # ms

prev_st = -1
curr_step = -1
activations = []

for st, sid in zip(spkt, spkid):
    if st < 150:
        continue

    if st > prev_st + space_between:
        curr_step += 1
        activations.append({ 'es': [], 'rest': [], 'min': spkt[-1] + 100, 'max': -1})
    activations[curr_step]['es' if sid in input_pop else 'rest'].append(sid)
    activations[curr_step]['min'] = min(st, activations[curr_step]['min'])
    activations[curr_step]['max'] = max(st, activations[curr_step]['max'])
    
    prev_st = st
    
assert len(activations) == 8000

In [ ]:
print('len', len(activations))
print(activations[2012])

In [ ]:
diffs = [act['max'] - act['min']  for act in activations]

In [ ]:
min(diffs), max(diffs)

In [ ]:
import matplotlib.pyplot as plt

es_acts = [len(act['es']) for act in activations]

plt.hist(es_acts)
plt.yscale('log')
plt.show()

In [ ]:
import os
import csv
import json
import pickle as pkl
from tqdm import tqdm

input_pop = list(range(0, 80))
space_between = 50.0 # ms

all_activations = []
for STEP_IDX in tqdm(range(20), total=20):
#     wdir = '../results/hpsearch-2021-09-06/best/1_run_168/evalmockAllStates/step_{}/'.format(STEP_IDX)
#     wdir = '../results/20210907-ES1500it/evalmockAllStates/step_{}/'.format(STEP_IDX)
    wdir = '../results/20210907-ES1500it/evalmockAllStates_atInit/step_{}/'.format(STEP_IDX)

    with open(os.path.join(wdir, 'sim.pkl'), 'rb') as f:
        sim = pkl.load(f)

    ts = []
    with open(os.path.join(wdir, 'ActionsRewards.txt')) as f:
        for row in csv.reader(f, delimiter='\t'):
            ts.append(float(row[0]))

    ts.append(ts[-1] + 50)

    spkt = sim['simData']['spkt']
    spkid = sim['simData']['spkid']

    prev_st = -1
    curr_step = -1
    activations = []

    for st, sid in zip(spkt, spkid):
        if st < 150:
            continue

        if st > prev_st + space_between:
            curr_step += 1
            activations.append({ 'es': [], 'rest': [], 'min': spkt[-1] + 100, 'max': -1})
        activations[curr_step]['es' if sid in input_pop else 'rest'].append(sid)
        activations[curr_step]['min'] = min(st, activations[curr_step]['min'])
        activations[curr_step]['max'] = max(st, activations[curr_step]['max'])

        prev_st = st

    assert len(activations) == 8000
    
    all_activations.extend(activations)

In [ ]:
print(len(all_activations))

In [ ]:
pop_left = {k: True for k in range(140, 160)}
pop_right = {k: True for k in range(160, 180)}

for act in tqdm(all_activations):
    moves = [0, 0, 0]
    for cid in act['rest']:
        moves[0 if cid in pop_left else (1 if cid in pop_right else 2)] += 1
    if moves[0] == 0 and moves[1] == 0:
        act['move'] = '0'
    elif moves[0] == moves[1]:
        act['move'] = 'u'
    elif moves[0] < moves[1]:
        act['move'] = 'r'
    elif moves[0] > moves[1]:
        act['move'] = 'l'
    else:
        assert False

In [ ]:
# es_all_activations = all_activations
# stdp_all_activations = all_activations
start_all_activations = all_activations

In [ ]:
from collections import Counter

for activations, name in [[stdp_all_activations, 'STDP'], [es_all_activations, 'ES']]:
    keys = ['0', 'u', 'l', 'r']
    keys_names = ['no EM activation', 'undecided', 'left move', 'right move']

    moves = Counter(''.join([act['move'] for act in activations]))
#     moves

    x = list(range(len(keys)))
    plt.figure(figsize=(10,10))
    plt.grid(axis='y', alpha=0.3)
    plt.bar(x, [moves[k] for k in keys])
    plt.xticks(x, keys_names)
    plt.ylim(0, max(moves.values()) + 5000)
    plt.xlabel('move result')
    plt.ylabel('count of states')
    plt.title('Distribution of {} result types on all states'.format(name))
    plt.show()

In [ ]:
import numpy as np

M = np.zeros((len(keys), len(keys)))

for act_stdp, act_es in zip(stdp_all_activations, es_all_activations):
    m1 = keys.index(act_stdp['move'])
    m2 = keys.index(act_es['move'])
    M[m1][m2] += 1
    
M

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.imshow(M)
plt.xticks(range(len(keys)), keys_names)
plt.yticks(range(len(keys)), keys_names)

# Loop over data dimensions and create text annotations.
for i in range(len(keys)):
    for j in range(len(keys)):
        text = plt.text(j, i, M[i, j],
                       ha="center", va="center", color="w")
        
plt.colorbar()

plt.ylabel('STDP choice')
plt.xlabel('ES choice')

plt.show()

In [ ]:
M[0]

In [ ]:
print('similar moves', sum(np.diag(M)))
print('percentage', sum(np.diag(M)) / np.sum(M))

In [ ]:
keys = ['0', 'u', 'l', 'r']
keys_names = ['no EM activation', 'undecided', 'left move', 'right move']

In [ ]:
import numpy as np
# for act_stdp, act_es in zip(stdp_all_activations, es_all_activations):

all_influences = {}
# for all_activations, name in [[stdp_all_activations, 'stdp'], [es_all_activations, 'es']]:
    
# all_activations = stdp_all_activations
all_activations = all_activations

# TODO: loop over all activations once only and create the matrix from that

influence = []
for cid in tqdm(range(200)):
    activated = np.zeros((len(keys)))
    total = np.zeros((len(keys)))
    for act in all_activations:
        if cid in act['es'] or cid in act['rest']:
            activated[keys.index(act['move'])] += 1
        total[keys.index(act['move'])] += 1
    influence.append(activated / total)

all_influences[name] = influence

In [ ]:
activated

In [ ]:
total

In [ ]:
from neurosim.tools.utils import _get_pop_name

sorted_min_ids = [('stimModES', 200),
 ('IML', 190),
 ('IM', 180),
 ('EM-RIGHT', 160),
 ('EM-LEFT', 140),
 ('IAL', 130),
 ('IA', 120),
 ('EA', 80),
 ('ES', 0)]

In [ ]:
es_c_stats = c_stats

In [ ]:
c_stats = []

for i, inf in enumerate(influence):
    power = inf[2:].sum()
    norm = inf[2:] / power
    left_pref = norm[0]
#     right_pref = norm[3]

    pop_name = _get_pop_name(i, sorted_min_ids)
    c_stats.append([i, pop_name, left_pref, power])


In [ ]:
sorted(c_stats, key=lambda x:x[3])

In [ ]:

indices = list(range(len(sorted_min_ids)-1, -1, -1))
pops = [sorted_min_ids[i][0] for i in indices]

plt.figure(figsize=(10,10))

for pop_name in pops:
    pop_stats = [(left_pref, power) for _,pname,left_pref,power in c_stats if pname == pop_name]
    xvals = [1.0-x for x,y in pop_stats]
    yvals = [y / 2.0 for x,y in pop_stats]
    plt.scatter(xvals, yvals, s=100.0)
    
plt.legend(pops[:-1])
plt.xlabel('Right Preference')
plt.ylabel('Power of influence')
plt.xlim(0, 1.0)
plt.xticks(np.arange(0, 1.0, 0.1))
plt.grid(alpha=0.2)
plt.title('ES model influence of each neuron based on all activation types from ES pop')
plt.show()

In [ ]:

indices = list(range(len(sorted_min_ids)-1, -1, -1))
pops = [sorted_min_ids[i][0] for i in indices]

plt.figure(figsize=(10,10))

for pop_name in pops:
    pop_stats = [(left_pref, power) for _,pname,left_pref,power in c_stats if pname == pop_name]
    xvals = [(0.5-x)*2 for x,y in pop_stats]
    yvals = [y / 2.0 for x,y in pop_stats]
    plt.scatter(xvals, yvals, s=100.0)
    
plt.legend(pops[:-1])
plt.xlabel('Right Preference')
plt.ylabel('Power of influence')
plt.xlim(0, 1.0)
plt.xticks(np.arange(-1, 1.0, 0.2))
plt.grid(alpha=0.2)
plt.title('STDP model influence of each neuron based on all activation types from ES pop')
plt.show()

In [ ]:
import matplotlib.pyplot as plt


indices = list(range(len(sorted_min_ids)-1, -1, -1))
pops = [sorted_min_ids[i][0] for i in indices]

plt.figure(figsize=(10,10))

for pop_name in pops:
    pop_stats = [(left_pref, power) for _,pname,left_pref,power in c_stats if pname == pop_name]
    xvals = [(0.5-x)*2 for x,y in pop_stats]
    yvals = [y / 2.0 for x,y in pop_stats]
    plt.scatter(xvals, yvals, s=100.0)
    
plt.legend(pops[:-1])
plt.xlabel('Right Preference')
plt.ylabel('Power of influence')
plt.xlim(0, 1.0)
plt.xticks(np.arange(-1, 1.0, 0.2))
plt.grid(alpha=0.2)
plt.title('Before Training model influence of each neuron based on all activation types from ES pop')
plt.show()

Previous, code for only 1 activation on ES

In [ ]:
tidx = 0 # index of the place in the step-wise time increment
curr_sid = None
curr_tidx = None

causal_graph = {}
curr_sid = None
curr_tidx = None

verbose = True

for st, sid in zip(spkt, spkid):
    if st < 150:
        continue
#     if st < 24216:
#         continue
    if verbose: print(st, sid)
    if st > ts[tidx]:
        if verbose: print('incr', ts[tidx])
        tidx += 1
    while st > ts[tidx]:
        if verbose: print('incr++', ts[tidx])
        curr_sid = None
        curr_tidx = None
        tidx += 1
    if curr_sid == None:
        curr_sid = sid
        curr_tidx = tidx
        if curr_sid in causal_graph:
            print('Stopping at {} of spkid {}'.format(st, sid))
            break
    if curr_sid not in causal_graph:
        causal_graph[curr_sid] = [[]]
    if tidx - curr_tidx == len(causal_graph[curr_sid]):
        causal_graph[curr_sid].append([])
    causal_graph[curr_sid][tidx - curr_tidx].append(sid)
    if verbose: print('curr_sid', curr_sid)
    

In [ ]:
causal_graph

In [ ]:
for i1,i2 in zip(sorted(list(causal_graph.keys())), range(80)):
    assert i1 == i2

In [ ]:
for st, sid in zip(spkt, spkid):
#     if st > 300:
        print(st, sid)

In [ ]:
spkid

In [ ]:
from neurosim.game_interface import GameInterface
import gym

env = gym.make('CartPole-v1')

class MockAIGame:
    observations = []
    rewards = []
    env = None

    def __init__(self, env):
        self.env = env
        
config = None
with open('../config.json') as f:
    config = json.load(f)

game = MockAIGame(env)
gi = GameInterface(game, config)

In [ ]:
import os
import csv
import json
import pickle as pkl

wdir = '../results/hpsearch-2021-09-06/best/1_run_168/evaluation_8/'

with open(os.path.join(wdir, 'sim.pkl'), 'rb') as f:
    sim = pkl.load(f)

ts = []
obs_fr = []
with open(os.path.join(wdir, 'ActionsRewards.txt')) as f:
    for row in csv.reader(f, delimiter='\t'):
        ts.append(float(row[0]))
        obs = json.loads(row[3])
        
        game.observations = [obs]
        f_r = gi.input_firing_rates()
        f_ind = (f_r > 0).nonzero()[0]
        obs_fr.append(f_ind)
        
ts.append(ts[-1] + 50)

spkt = sim['simData']['spkt']
spkid = sim['simData']['spkid']

In [ ]:
tidx = 0 # index of the place in the step-wise time increment

es_spks_at_times = {}
for st, sid in zip(spkt, spkid):
    if sid < 80:
        while st > ts[tidx]:
            tidx += 1
#         print('\t'.join([str(k) for k in [ts[tidx-1], round(st, 2), ts[tidx], sid]]))
        if tidx not in es_spks_at_times:
            es_spks_at_times[tidx] = []
        es_spks_at_times[tidx].append(sid)
        es_spks_at_times[tidx] = sorted(es_spks_at_times[tidx])
        
print(len(es_spks_at_times))

In [ ]:
for idx, (k,v) in enumerate(es_spks_at_times.items()):
    print('---', k, [int(s) for s in sorted(list(set(v)))], obs_fr[idx-1])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import numpy as np

plt.figure(figsize=(10,10))
# plt.boxplot()
d = [len(set(v)) for k,v in es_spks_at_times.items()][3:]
t_d = sum(d)
plt.hist(d, weights=np.ones(len(d)) / len(d))

plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.xlabel('number of distinct ES neurons firing in a timestep')
plt.ylabel('percentage of timesteps')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import numpy as np

plt.figure(figsize=(10,10))
# plt.boxplot()
d = [len(set(v)) for k,v in es_spks_at_times.items()][3:]
# d = [len(v) for k,v in es_spks_at_times.items()][3:]
t_d = sum(d)
plt.hist(d, weights=np.ones(len(d)) / len(d))

plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.xlabel('number of distinct ES neurons firing in a timestep')
plt.ylabel('percentage of timesteps')
plt.show()

In [ ]:
[len(set(v)) for k,v in es_spks_at_times.items()][4:]

In [ ]:
len([s for s in [len(set(v)) for k,v in es_spks_at_times.items()] if s == 5]) / len([len(set(v)) for k,v in es_spks_at_times.items()])